# **Лабораторна робота №1**
## з дисципліни **"Методи оптимізації та дослідження операцій"**

##### *За темою:* **"Методи оптимізації першого першого порядку"**

<div style="display: flex;">

<div style="flex: 1;">
    <i>Виконавці роботи:</i><br>
    бригада №21
    <br><br>
    <i><b>
    Баштовий Іван<br>
    Гавлицький Іван<br>
    Харитонов Олександр<br>
    Ходаковський Артур
    </i></b>
</div>

<div style="flex: 1;">
<i>Прийняли:</i>
<br><br><br><br>
<i><b>Яковлєва А. П.</i></b><br>
<i><b>Спекторський І. Я.</i></b>
</div>

</div>

In [1]:
import numpy as np
import pandas as pd
import sympy as sp
import matplotlib.pyplot as plt

np.random.seed(0)

## Варіант 15


In [2]:
def func(x, y=0, z=0):
    return 12*x**2 + 18*y**2 +  3*z**2 - 0.01*x*z + x - y

## **Хід роботи**

### **Дослідимо функцію аналітично**

In [3]:
symbols = sp.symbols('x y z')
f = func(*symbols)

f

12*x**2 - 0.01*x*z + x + 18*y**2 - y + 3*z**2

### Матриця Гессе

In [4]:
def hessian_matrix(f, symbols):
    n = len(symbols)
    hessian = sp.zeros(n, n)
    
    for i, symbol1 in enumerate(symbols):
        for j, symbol2 in enumerate(symbols):
            hessian[i, j] = sp.diff(sp.diff(f, symbol1), symbol2).evalf(2)

    return hessian

In [5]:
hessian = hessian_matrix(f, symbols)

print("Матриця Гессе:")
hessian

Матриця Гессе:


Matrix([
[ 24.0,    0, -0.01],
[    0, 36.0,     0],
[-0.01,    0,   6.0]])

### Точка мінімуму

In [6]:
f_grad = sp.derive_by_array(f, symbols)

print('Похідні:')
f_grad

Похідні:


[24*x - 0.01*z + 1, 36*y - 1, -0.01*x + 6*z]

In [7]:
def find_min(func, symbols):
    f = func(*symbols)
    f_grad = sp.derive_by_array(f, symbols)

    equations = [sp.Eq(diff, 0) for diff in f_grad]

    res = sp.solve(equations, symbols)
    print(f'Точка, в якій досягається мінімум: {res}')


    if isinstance(res, dict):  # Checking if the result is a dictionary
        f_min = func(*res.values())
        print("Значення мінімуму функції:", f_min)
    elif isinstance(res, list) and len(res) > 0:  # If res is a list with solutions
        # Accessing the first solution as an example
        first_solution = res[0]
        if isinstance(first_solution, tuple):
            f_min = func(*first_solution)
            print("Значення мінімуму функції за першим розв'язком:", f_min)
        else:
            print("Не вдалося обчислити мінімум: розв'язки не у очікуваному форматі.")
    else:
        print("Розв'язок не знайдено або функція не має мінімуму.")

In [8]:
find_min(func, symbols)

Точка, в якій досягається мінімум: {x: -0.0416666956018719, y: 0.0277777777777778, z: -6.94444926697866e-5}
Значення мінімуму функції: -0.0347222366898249


### Обумовленість

In [9]:
eigenvalues = hessian.eigenvals()

print("Власні числа матриці:")
for eigenvalue in eigenvalues:
    print(eigenvalue)

Власні числа матриці:
36.0000000000000
24.0000055494893
5.99999445055437


In [10]:
try:
    print(f'Перевірка на обумовленість: d/D = {min(eigenvalues)/max(eigenvalues)}')
except Exception as e:
    print(f"Error")

Перевірка на обумовленість: d/D = 0.166666512515399


### **Градієнтний спуск**

In [11]:
class GradientDescending(object):
    def __init__(self, func, symbols, step_size_method='steepest', step_size_method_args={}):
        self.func = func
        self.symbols = symbols
        self.f = func(*symbols)

        self.step_size_method = self._get_step_size_func(step_size_method)
        self.step_size_method_args = step_size_method_args

        self.f_grad = sp.derive_by_array(self.f, symbols)
        self.f_hessian = self._hessian_matrix(self.f, symbols)

        self.v_i = 0 #RMSprop

    
    def fit(self, initial_point, eps=1e-5):
        self.points = [initial_point]
        self.grads = []
        self.learning_rates = []

        while True:
            self.grads.append(self._symbols_obj_at_point(self.f_grad, self.points[-1]))
            self.learning_rates.append(self.step_size_method(self.points[-1], **self.step_size_method_args))
            new_point = self.points[-1] - self.learning_rates[-1] * self.grads[-1]

            if np.linalg.norm(self.points[-1] - new_point) < eps:
                break

            self.points.append(new_point)

        return self
    

    def get_min_point(self):
        return self.points[-1]

    def get_min_value(self):
        return self.func(*self.points[-1])
    
    def get_history_df(self, round_n):
        history = {
            'point' : [tuple(point.round(round_n)) for point in self.points],
            'grad' : [tuple(grad.round(round_n)) for grad in self.grads],
            'learning_rate' : self.learning_rates,
            'func' : [self.func(*point) for point in self.points]
        }

        return pd.DataFrame(history)

    def _hessian_matrix(self, f, symbols):
        n = len(symbols)
        hessian = sp.zeros(n, n)
        
        for i, symbol1 in enumerate(symbols):
            for j, symbol2 in enumerate(symbols):
                hessian[i, j] = sp.diff(sp.diff(f, symbol1), symbol2).evalf(2)

        return hessian
    

    def _symbols_obj_at_point(self, symbols_obj, point):
        temp = symbols_obj.subs(dict(zip(symbols, point)))
        return np.array(temp, dtype=float)
    


    def _get_step_size_func(self, step_size_method):
        temp = {
            'splitting' : self._step_splitting_step_size,
            'steepest' : self._steepest_descent_step_size
        }

        return temp[step_size_method]

    def _step_splitting_step_size(self, point, alpha=1, lmbda=0.5, beta=0.1):
        grad = self.grads[-1]
        while func(*(point - alpha*grad)) > func(*point) - alpha * beta * np.dot(grad, grad):
            alpha *= lmbda

        return alpha
    
    def _steepest_descent_step_size(self, point):
        A = self._symbols_obj_at_point(self.f_hessian, point)
        grad = self.grads[-1]

        learning_rate = (grad @ grad) / (grad.T @ A @ grad)
        return learning_rate

In [12]:
def collect_results(gradient_descent_args, starting_points):
    results = {}

    for i, point in enumerate(starting_points):
        print(f'Starting point: {tuple(point)}')

        gradient_descent = GradientDescending(**gradient_descent_args).fit(point)
        table = gradient_descent.get_history_df(2)

        print(table.to_string())

        results[f'Starting Point {i+1}'] = table.shape[0]
        print('\n'*3)

    # Display iteration counts for each starting point
    print("Iterations required for convergence from different starting points:")
    for key, value in results.items():
        print(f"{key}: {value} iterations")

#### **Метод дроблення**

In [13]:
d = len(symbols)

starting_points = np.array([
    [-1]*d,
    [0]*d,
    [10]*d
])

collect_results(
    starting_points=starting_points,
    gradient_descent_args={
        'func' : func,
        'symbols' : symbols,
        'step_size_method' : 'splitting'
    }
)

Starting point: (-1, -1, -1)
                   point                    grad  learning_rate       func
0           (-1, -1, -1)  (-22.99, -37.0, -5.99)        0.03125  32.990000
1   (-0.28, 0.16, -0.81)    (-5.75, 4.62, -4.87)        0.06250   2.932674
2   (0.08, -0.13, -0.51)    (2.87, -5.78, -3.05)        0.06250   1.375822
3    (-0.1, 0.23, -0.32)     (-1.44, 7.23, -1.9)        0.03125   1.036076
4    (-0.06, 0.0, -0.26)    (-0.36, -0.9, -1.55)        0.12500   0.178980
5   (-0.01, 0.12, -0.06)     (0.72, 3.16, -0.39)        0.03125   0.127339
6   (-0.03, 0.02, -0.05)     (0.18, -0.4, -0.31)        0.06250  -0.023642
7   (-0.05, 0.04, -0.03)     (-0.09, 0.49, -0.2)        0.03125  -0.027947
8   (-0.04, 0.03, -0.03)   (-0.02, -0.06, -0.16)        0.12500  -0.032534
9   (-0.04, 0.03, -0.01)     (0.04, 0.22, -0.04)        0.03125  -0.033899
10  (-0.04, 0.03, -0.01)    (0.01, -0.03, -0.03)        0.06250  -0.034622
11   (-0.04, 0.03, -0.0)    (-0.01, 0.03, -0.02)        0.06250  -0.034

#### **Метод найшвидшого спуску**

In [14]:
d = len(symbols)

starting_points = np.array([
    [-1]*d,
    [0]*d,
    [10]*d,
    [30]*d,
    [-4, 5, 27]
])

collect_results(
    starting_points=starting_points,
    gradient_descent_args={
        'func' : func,
        'symbols' : symbols,
        'step_size_method' : 'steepest'
    }
)

Starting point: (-1, -1, -1)
                    point                    grad  learning_rate       func
0            (-1, -1, -1)  (-22.99, -37.0, -5.99)       0.031093  32.990000
1    (-0.29, 0.15, -0.81)    (-5.84, 4.42, -4.88)       0.046560   2.931909
2   (-0.01, -0.06, -0.59)    (0.68, -2.99, -3.52)       0.053544   1.130730
3      (-0.05, 0.1, -0.4)     (-0.2, 2.77, -2.39)       0.043087   0.547972
4    (-0.04, -0.01, -0.3)    (0.01, -1.53, -1.77)       0.053222   0.258985
5     (-0.04, 0.07, -0.2)      (-0.0, 1.4, -1.21)       0.043083   0.113516
6    (-0.04, 0.01, -0.15)    (-0.0, -0.77, -0.89)       0.053222   0.040096
7     (-0.04, 0.05, -0.1)     (-0.0, 0.71, -0.61)       0.043083   0.003040
8    (-0.04, 0.02, -0.08)    (-0.0, -0.39, -0.45)       0.053222  -0.015663
9    (-0.04, 0.04, -0.05)     (-0.0, 0.36, -0.31)       0.043083  -0.025103
10   (-0.04, 0.02, -0.04)     (-0.0, -0.2, -0.23)       0.053222  -0.029867
11   (-0.04, 0.03, -0.03)     (-0.0, 0.18, -0.15)       0.0

#### **Тестова функція**

In [15]:
# функция Розенброка
def test_func(x, y):
    return 10_000*(y - x**2)**2 + 100*(1 - x)**2

test_symbols = sp.symbols('x y')

In [16]:
find_min(test_func, test_symbols)

Точка, в якій досягається мінімум: [(1, 1)]
Значення мінімуму функції за першим розв'язком: 0


In [17]:
d = len(test_symbols)

starting_points = np.array([
    [-1]*d,
    [0]*d,
    [30]*d
])

collect_results(
    starting_points=starting_points,
    gradient_descent_args={
        'func' : test_func,
        'symbols' : test_symbols,
        'step_size_method' : 'steepest'
    }
)

Starting point: (-1, -1)
               point                    grad  learning_rate          func
0           (-1, -1)    (-80400.0, -40000.0)       0.000006  40400.000000
1     (-0.51, -0.76)  (-21088.58, -20349.06)       0.000016  10580.338392
2     (-0.17, -0.43)    (-3364.32, -9157.51)       0.000041   2233.601426
3     (-0.03, -0.06)     (-282.92, -1128.88)       0.000051    138.720557
4       (-0.02, 0.0)        (-202.39, 38.05)       0.001816    103.935020
5      (0.35, -0.07)     (2488.93, -3761.15)       0.000031    396.140447
6       (0.27, 0.05)         (98.54, -451.8)       0.000042     58.318147
7       (0.27, 0.07)        (-130.9, -29.69)       0.000492     53.839967
8       (0.33, 0.08)       (201.63, -507.01)       0.000036     51.205601
9        (0.32, 0.1)       (-105.53, -45.99)       0.000999     45.810891
10      (0.43, 0.15)       (497.84, -713.45)       0.000029     45.336426
11      (0.41, 0.17)        (-63.89, -64.12)       0.002120     34.362853
12       (0.5

C:\Users\igali\AppData\Local\Temp\ipykernel_12116\2062769561.py:2: RuntimeWarning: overflow encountered in scalar multiply
  return 10_000*(y - x**2)**2 + 100*(1 - x)**2


               point                         grad  learning_rate          func
0           (30, 30)  (1044005800.0, -17400000.0)   9.360315e-09 -1.020850e+09
1     (20.23, 30.16)  (306657023.16, -7580001.91)   2.086369e-08  1.436448e+09
2     (13.83, 30.32)   (89034136.35, -3218835.56)   4.595015e-08  2.590390e+08
3      (9.74, 30.47)   (25077727.03, -1287446.69)   9.821563e-08  4.144561e+07
4       (7.28, 30.6)     (6502553.07, -446786.35)   1.942608e-07  4.994389e+06
5      (6.01, 30.68)     (1315843.06, -109343.49)   3.193225e-07  3.014124e+05
6      (5.59, 30.72)       (125339.86, -11124.44)   3.927541e-07  5.202710e+03
7      (5.54, 30.72)            (1726.37, -73.77)   4.040746e-07  2.064044e+03
8      (5.54, 30.72)                (5.18, 81.49)   5.719441e-04  2.063441e+03
9      (5.54, 30.67)           (3059.53, -194.21)   4.039499e-07  2.061528e+03
10     (5.54, 30.67)                 (8.6, 81.17)   1.616566e-03  2.059628e+03
11     (5.52, 30.54)            (-4983.93, 533.0)   

## **Спроби ще щось написати**

### **Ярний метод**

In [18]:
class RavineGradientDescending(GradientDescending):  
    def fit(self, point_1, point_2, revine_step=1, eps=1e-5):
        self.points = [point_1, point_2]
        self.ravine_points = [super().fit(point, eps=eps).get_min_point() for point in self.points]
        self.ravine_values = [self.func(*point) for point in self.ravine_points]

        while np.linalg.norm(self.ravine_points[-1] - self.ravine_points[-2]) < eps:
            new_point = self.ravine_points[-1] - self.ravine_points[-2]
            new_point /= np.linalg.norm(new_point)
            new_point *= revine_step*np.sign(self.ravine_values[-1] - self.ravine_values[-2])
            
            self.points.append(new_point)
            self.ravine_points.append(super().fit(new_point, eps=eps).get_min_point())

        return self
    
    def get_history_df(self, round_n):
        history = {
            'point' : [tuple(point.round(round_n)) for point in self.points],
            'ravine_point' : [tuple(point.round(round_n)) for point in self.ravine_points],
            'ravine_func' : self.ravine_values
        }

        return pd.DataFrame(history)

In [19]:
def plot_contour(f, symbols, x_range=(-5, 5), y_range=(-5, 5), levels=60):
    x_vals = np.linspace(x_range[0], x_range[1], 100)
    y_vals = np.linspace(y_range[0], y_range[1], 100)
    X, Y = np.meshgrid(x_vals, y_vals)

    f = sp.lambdify(symbols, f, 'numpy')
    Z = f(X, Y)

    plt.contour(X, Y, Z, levels=levels)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Contour plot of f(x, y)')
    plt.grid(True)
    plt.show()

## **Висновки**
##### Ми освоїти методику та зрозуміти сенс обчислення ковзного середнього та часткової автокореляційної функції.